In [1]:
import pandas as pd

import urllib.request
from PIL import Image

In [2]:
train = pd.read_csv("data/train.csv")
train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4GB,500GB HDD,AMD Radeon R5,Windows 10,2.4kg,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8GB,256GB SSD,Intel Iris Plus Graphics 640,Windows 10,1.23kg,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,3kg,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.26kg,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8GB,1TB HDD,AMD Radeon R4,Windows 10,2.8kg,519.0


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [4]:
train["Ram"] = train["Ram"].str.rstrip("GB")
train["Memory"] = train["Memory"].str.replace("GB","")
train["Weight"] = train["Weight"].str.rstrip("kg")

train["Ram"] = train["Ram"].astype("int64")
train["Weight"] = train["Weight"].astype("float64")

train.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1104,Acer,Aspire ES1-523,Notebook,15.6,1366x768,AMD A8-Series 7410 2.2GHz,4,500 HDD,AMD Radeon R5,Windows 10,2.40,387.0
1,114,Dell,XPS 13,Ultrabook,13.3,Quad HD+ / Touchscreen 3200x1800,Intel Core i7 7560U 2.4GHz,8,256 SSD,Intel Iris Plus Graphics 640,Windows 10,1.23,1379.0
2,172,Acer,Aspire A517-51G,Notebook,17.3,IPS Panel Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8,256 SSD,Nvidia GeForce MX150,Windows 10,3.00,854.0
3,918,HP,Elitebook 820,Netbook,12.5,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,512 SSD,Intel HD Graphics 620,Windows 10,1.26,1483.0
4,447,Lenovo,IdeaPad 320-15AST,Notebook,17.3,1600x900,AMD A6-Series 9220 2.5GHz,8,1TB HDD,AMD Radeon R4,Windows 10,2.80,519.0


In [6]:
train["SDD"] = 0
train["HDD"] = 0
train["Other"] = 0.0

for mem in range(train.shape[0]):
    splitMem = str(train.iloc[mem]["Memory"]).split("+")
    for i in splitMem:
        if i.count("SSD") > 0:
            i = i.rstrip(" ").lstrip(" ").rstrip("SSD").rstrip(" ").lstrip(" ")
            if i.count("TB"):
                i = i.rstrip("TB")
                i = str(int(i)*1024)
                
            train.loc[mem,"SDD"] = int(i)
        elif i.count("HDD") > 0:
            i = i.rstrip(" ").lstrip(" ").rstrip("HDD").rstrip(" ").lstrip(" ")
            if i.count("TB"):
                i = i.rstrip("TB")
                i = str(int(i)*1024)
                
            train.loc[mem,"HDD"] = int(i)
        else:
            i = i.replace("Flash Storage","").replace("Hybrid","").rstrip(" ").lstrip(" ")
            if i.count("TB"):
                i = i.rstrip("TB")
                i = float(i)*1024
                
            train.loc[mem,"Other"] = float(i)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    int64  
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    float64
 12  Price_euros       912 non-null    float64
 13  SDD               912 non-null    int64  
 14  HDD               912 non-null    int64  
 15  Other             912 non-null    float64
dtypes: float64(4), int64(4), object(8)
memory us

In [8]:
X = train[["Inches", "Ram", "Weight","SDD","HDD"]]
y = train["Price_euros"]

Modelo

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [13]:
model.fit(X_train,y_train)

LinearRegression()

In [14]:
scores = -cross_val_score(model, X_train, y_train, cv=10, scoring="neg_root_mean_squared_error")

np.mean(scores)

402.1974790244654

# Datos generales

In [15]:
model.fit(X,y)

LinearRegression()

In [16]:
scores = -cross_val_score(model, X, y, cv=200, scoring="neg_root_mean_squared_error")

np.mean(scores)

372.56592880232245

In [17]:
pred = model.predict(X)
pred

array([ 583.65876082, 1195.95144062, 1207.65554327, 1573.55260954,
        817.3356071 , 1514.23370416, 1221.41972703, 1054.25499858,
       1195.95144062, 1059.62725396, 1800.78620943,  898.21024577,
       1195.95144062, 1206.18173573,  535.35809596, 1206.18173573,
       1124.89207274,  583.65876082,  907.09160495, 1179.62659888,
       2097.57201887, 1976.42369755,  937.64744634,  889.41891225,
       1178.41379186,  549.97280326, 1556.30972229, 1171.72972221,
       1211.18943192,  774.86951824, 1204.720265  , 1212.65090265,
        554.42934622, 1801.76711304, 1638.87545272, 1789.82668348,
       1250.64914163,  535.43022673,  554.42934622, 1768.89485389,
        554.42934622, 2180.28632747, 3497.2618485 ,  595.42111277,
        617.88659894, 1195.70277691,  453.7767356 ,  888.20610523,
        485.66809114,  671.99207894, 1521.23442477,  701.79598174,
       1747.44403231,  554.42934622, 1185.72114551, 2776.15459475,
        608.45766544, 1217.87350157, 1220.79644303, 1214.11237

In [18]:
test = pd.read_csv("data/test.csv")
test.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.25kg
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4GB,32GB HDD,Intel HD Graphics 400,Windows 10,1.4kg
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2GB,32GB Flash Storage,Intel HD Graphics,Windows 10,1.4kg
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8GB,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65kg
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2GB,32GB Flash Storage,Intel HD Graphics 500,Windows 10,1.1kg


In [19]:
test["Ram"] = test["Ram"].str.rstrip("GB")
test["Memory"] = test["Memory"].str.replace("GB","")
test["Weight"] = test["Weight"].str.rstrip("kg")

test["Ram"] = test["Ram"].astype("int64")
test["Weight"] = test["Weight"].astype("float64")

test.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,750,Toshiba,Tecra X40-D-10G,Notebook,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8,256 SSD,Intel HD Graphics 620,Windows 10,1.25
1,726,Mediacom,Smartbook 142,Notebook,14.0,IPS Panel Full HD 1920x1200,Intel Atom Z8350 1.92GHz,4,32 HDD,Intel HD Graphics 400,Windows 10,1.40
2,633,Mediacom,SmartBook 140,Notebook,14.0,Full HD 1920x1080,Intel Atom x5-Z8350 1.44GHz,2,32 Flash Storage,Intel HD Graphics,Windows 10,1.40
3,363,Dell,Inspiron 7577,Gaming,15.6,Full HD 1920x1080,Intel Core i5 7300HQ 2.5GHz,8,1TB HDD,Nvidia GeForce GTX 1050,Windows 10,2.65
4,319,Asus,VivoBook Flip,2 in 1 Convertible,11.6,Touchscreen 1366x768,Intel Celeron Dual Core N3350 1.1GHz,2,32 Flash Storage,Intel HD Graphics 500,Windows 10,1.10


In [24]:
test["SDD"] = 0
test["HDD"] = 0
test["Other"] = 0.0

for mem in range(test.shape[0]):
    splitMem = str(test.iloc[mem]["Memory"]).split("+")
    for i in splitMem:
        if i.count("SSD") > 0:
            i = i.rstrip(" ").lstrip(" ").rstrip("SSD").rstrip(" ").lstrip(" ")
            if i.count("TB"):
                i = i.rstrip("TB")
                i = str(int(i)*1024)
                
            test.loc[mem,"SDD"] = int(i)
        elif i.count("HDD") > 0:
            i = i.rstrip(" ").lstrip(" ").rstrip("HDD").rstrip(" ").lstrip(" ")
            if i.count("TB"):
                i = i.rstrip("TB")
                i = float(i)*1024
                
            test.loc[mem,"HDD"] = float(i)
        else:
            i = i.replace("Flash Storage","").replace("Hybrid","").rstrip(" ").lstrip(" ")
            if i.count("TB"):
                i = i.rstrip("TB")
                i = float(i)*1024
                
            test.loc[mem,"Other"] = float(i)

In [25]:
test_pred = test[["Inches", "Ram", "Weight","SDD","HDD"]]

In [26]:
predictions_submit = model.predict(test_pred)

# Creación de CSV

In [27]:
sample = pd.read_csv("data/sample_submission.csv")

In [28]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})

In [29]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [30]:
chequeator(submission)

You're ready to submit!
